In [ ]:
!pip -q install langchain openai tiktoken langchain-community rank_bm25 sentence-transformers chromadb httpx==0.27.2 pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 17.9 MB/s eta 0:00:00


In [ ]:
import tiktoken
import openai
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers  import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.vectorstores import FAISS

In [ ]:
!wget https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf

--2025-02-28 08:09:28--  https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf
Resolving wdr.ubion.co.kr (wdr.ubion.co.kr)... 61.100.182.43
Connecting to wdr.ubion.co.kr (wdr.ubion.co.kr)|61.100.182.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1253369 (1.2M) [application/pdf]
Saving to: ‘gsat_170823.pdf’

gsat_170823.pdf     100%[===================>]   1.19M  2.16MB/s    in 0.6s    

2025-02-28 08:09:30 (2.16 MB/s) - ‘gsat_170823.pdf’ saved [1253369/1253369]



In [ ]:
## pdf 파일로드 하고 쪼개기
loader = PyPDFLoader('https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf')
pages = loader.load_and_split()
print(len(pages))

27


In [ ]:
model_huggingface = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')

<ipython-input-4-029e6670ad57>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  model_huggingface = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.w

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
## chunk로 쪼개기
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(pages)

In [ ]:
len(texts)

69

In [ ]:
print(texts[0])

page_content='2
01 삼성전자 기업분석
(Samsung Electronics Co., Ltd)
Ⅰ 기업 일반 
1  기업개요
1) 기업소개 
본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)
사업분야 삼성그룹의 대표 기업으로 휴대폰, 정보통신기기, 반도체, TV 등을 생산 판매하는 제조업체
홈페이지 www.samsung.com/sec 구분 전기전자 대기업  
설립일 1961년 07월 01일 대표이사 권오현 
총자산1) 244조 매출액2) 200조
임직원수 95,374명 
∙ 1975년 1월 주식시장 상장
∙ 1984년 2월 삼성전자공업주식회사->삼성전자주식회사로 사명 변경 
∙ CE(Consumer Electronics), IM(Information technology & Mobile communications), DS(Device Solutions) 
3개의 부문으로 나누어 독립 경영.
부문 제품
CE TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등' metadata={'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27, 'page': 0, 'page_label': '1'}


## BM-25

In [ ]:
bm25_retriever = BM25Retriever.from_documents(texts)
bm25_retriever.k = 2

In [ ]:
# BM25 리트리버만 사용하는 경우
docs = bm25_retriever.get_relevant_documents("구글")

for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content.replace('\n',' '))
    print(len(i.page_content.replace('\n',' ')))
    print("*"*30)

{'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27, 'page': 10, 'page_label': '11'}
:
물론 글로벌 경쟁사인 구글 및 애플에 비해서도 5~10년 뒤처졌다. 글로벌 전장업체인 하만  인수는 이런 간격을 단번에 좁히고 뒤집겠다는 ‘한 수’다. 커넥티드카(무선인터넷으로 외부와  연결되는 자동차) 솔루션 및 인포테인먼트(정보+오락) 분야의 대표 기업인 하만과 반도체 디 스플레이 정보기술(IT) 스마트폰 등의 분야에서 앞선 기술을 가진 삼성이 손을 잡으면 단숨 에 스마트카 전장 부문에서 대표적인 1차 협력사(티어1) 지위를 차지할 수 있을 것이라는 게  두 회사가 내놓은 청사진이다. 삼성 + 하만, 스마트카 시대 주도  디네시 팔리월 하만 최고경영자(CEO)와 손영권 삼성전자 전략혁신센터장(사장), 박종환 삼 성전자 전장사업팀장(부사장)은 21일 서울 서초동 삼성사옥에서 기자간담회를 하고 두 회사 의 합병을 통해 기대할 수 있는 효과와 미래 전략을 공개했다 . 지난 14일 합병 결정을 발표 한 지 1주일 만이다.
463
******************************
{'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source

## 임베딩

In [ ]:
# 벡터 DB의 임베딩으로는 오픈소스 임베딩을 사용
chroma_vector = Chroma.from_documents(texts, model_huggingface)
chroma_retriever = chroma_vector.as_retriever(search_kwargs={'k':2})

In [ ]:
# 크로마 리트리버만 사용하는 경우
docs = chroma_retriever.invoke("구글 바드는?")
for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content.replace('\n',' '))
    print(len(i.page_content.replace('\n',' ')))
    print("*"*30)

{'creationdate': '2017-08-16T00:21:02-08:00', 'creator': 'nPDF (pdftk 1.41)', 'moddate': '2017-08-16T00:21:02-08:00', 'page': 19, 'page_label': '20', 'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27}
:
합성어로, 차량 내 자동차가 무선 네트워크과 직접 연결되어 차가 하나의 커다란 스마트폰이 될 수  있는 시스템이다. ∙AP : 스마트폰·디지털 TV 등에 사용되는 비메모리 반도체로 일반 컴퓨터의 중앙처리장치(CPU)와 같은  역할을 한다. ∙ 하이엔드 : 오디오 시장에서 주로 사용되는 용어로, 제품군 가운데 가장 뛰어난 성능을 가진 제품을  가리킨다. ∙낸드플래시 : 플래시 메모리의 한 형태로 전원이 없는 상태에서도 데이터를 계속 저장할 수 있으며 데이터 를 자유롭게 저장·삭제할 수 있다. 2002년부터 세계 1위를 유지하고 있다. ∙스마트워크 : 스마트 워크란 종래의 사무실 근무를 벗어나 언제 어디서나 효율적으로 일할 수 있는 업무개 념을 뜻한다. ∙잉여현금흐름 : 기업에 현금이 얼마나 순유입 되었는지를 나타내는 지표로 기업이 사업 활동에서 벌어들인  현금에서 각종 비용과 세금, 설비투자 등을 빼고 남은 잔여 현금흐름을 말한다. ∙ 포터블 : 사람이 휴대하기 편리하게 생산된 기기
499
******************************
{'creationdate': '2017-08-16T00:21:02-08:00', 'creator': 'nPDF (pdftk 1.41)', 'moddate': '2017-08-16T00:21:02-08:00', 'page': 14, 'page_label': '15', 

## 앙상블

### EnsembleRetriever 작동 과정 (상세 예시)

먼저 두 검색기가 있고, 각각 k=4개의 문서를 반환한다고 가정해보겠습니다:

#### 1. 개별 검색기 결과 (각 검색기의 상위 4개 문서)
```
BM25 검색기 결과:
- 문서 A: 점수 0.9
- 문서 B: 점수 0.8
- 문서 C: 점수 0.7
- 문서 D: 점수 0.6

임베딩 검색기 결과:
- 문서 A: 점수 0.95 (BM25와 중복)
- 문서 E: 점수 0.85
- 문서 F: 점수 0.75
- 문서 B: 점수 0.65 (BM25와 중복)
```
여기서 문서 A와 B는 두 검색기 모두에서 반환되었습니다.

#### 2. 가중치 적용 (각 검색기에 0.5씩 가중치 부여)

각 검색기의 점수에 가중치 0.5를 곱합니다:
```
BM25 검색기:
- 문서 A: 0.9 × 0.5 = 0.45
- 문서 B: 0.8 × 0.5 = 0.40
- 문서 C: 0.7 × 0.5 = 0.35
- 문서 D: 0.6 × 0.5 = 0.30

임베딩 검색기:
- 문서 A: 0.95 × 0.5 = 0.475
- 문서 E: 0.85 × 0.5 = 0.425
- 문서 F: 0.75 × 0.5 = 0.375
- 문서 B: 0.65 × 0.5 = 0.325
```


#### 3. 중복 문서 점수 합산

동일한 문서가 여러 검색기에서 나온 경우, 가중치가 적용된 점수들을 합산합니다:
```
- 문서 A: 0.45 (BM25) + 0.475 (임베딩) = 0.925
- 문서 B: 0.40 (BM25) + 0.325 (임베딩) = 0.725
- 문서 C: 0.35 (BM25만) = 0.35
- 문서 D: 0.30 (BM25만) = 0.30
- 문서 E: 0.425 (임베딩만) = 0.425
- 문서 F: 0.375 (임베딩만) = 0.375
```


#### 4. 점수 기준 정렬 및 최종 결과 반환
```
모든 문서를 최종 점수 기준으로 내림차순 정렬하고, 상위 k개(예: k=4)를 반환합니다:

최종 정렬:
1. 문서 A: 0.925 (두 검색기 모두에서 높은 점수)
2. 문서 B: 0.725 (두 검색기 모두에서 등장)
3. 문서 E: 0.425 (임베딩 검색기에서만)
4. 문서 F: 0.375 (임베딩 검색기에서만)
5. 문서 C: 0.35 (BM25 검색기에서만)
6. 문서 D: 0.30 (BM25 검색기에서만)

최종 반환 결과(k=4): 문서 A, B, E, F
```

### 핵심 포인트
1. **중복 문서 강화**: 두 검색기 모두에서 나온 문서(A, B)는 점수가 합산되어 순위가 높아집니다. 이는 여러 방식으로 관련성이 확인된 문서가 우선시됨을 의미합니다.

2. **다양한 결과 포함**: 각 검색기의 고유한 강점을 활용하여 키워드 매칭(BM25)과 의미적 유사성(임베딩) 모두에서 관련성 높은 문서를 포함합니다.

3. **가중치 영향**: 만약 BM25에 더 높은 가중치(예: 0.7)를 부여한다면, BM25 결과가 최종 순위에 더 큰 영향을 미치게 됩니다.

In [ ]:
ensemble_retriever = EnsembleRetriever(
                    retrievers = [bm25_retriever, chroma_retriever], weight = {0.5,0.5})

docs = ensemble_retriever.invoke("삼성전자의 사업 영역은?")

In [ ]:
# 문서의 개수는 총 4개이다.
len(docs)

4

## 앙상블 결과

In [ ]:
for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content.replace('\n',' '))
    print(len(i.page_content.replace('\n',' ')))
    print("*"*30)

{'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27, 'page': 10, 'page_label': '11'}
:
12 Q2 삼성전자의 Harman사 인수에 대해 어떻게 생각하십니까 ? A  삼성전자가 최근 80억달러(약 9조)에 미국의 전장업체(자동차 전자기기에 대한 사업)인 Harman 사를  인수하기로 결정했습니다. 하만사는 세계적으로 자동차용 인포테인먼트와 텔레매틱스 시장에서 점유 율 1~2위를 달리고 있으며 매출도 상당합니다. 이번 삼성전자의 하만 인수는 모바일 및 가전 시장에서  나아가 Connected Car 시장 진입에 진출했다는 것에 큰 의의가 있습니다. 향후 10년 시장 내 Connected  Car의 비중은 90%에 육박할 전망입니다. LG전자 역시 자동차부품(VC) 사업 부문에 있어 상당한 경쟁  위협 요인가 될 것으로 내다봤습니다. <관련기사> 전장사업 강자 하만과 손잡은 삼성…스마트카 ‘티어1’ 노린다 (한국경제. 2016-11-21) 삼성전자는 자동차 전장(電裝)사업에서 후발 주자다. 2005년 전장사업을 시작한 LG전자는
474
******************************
{'creationdate': '2017-08-16T00:21:02-08:00', 'creator': 'nPDF (pdftk 1.41)', 'moddate': '2017-08-16T00:21:02-08:00', 'page': 9, 'page_label': '10', 'producer': 'itext-paulo-1

## 하이브리드 서치

### 두 검색 방식의 차이점: 간단 예시

**문서 목록:**
- A: "삼성전자 반도체"
- B: "삼성전자 스마트폰"
- C: "반도체 기술"
- D: "삼성 신제품"

**검색어:** "삼성 반도체"

**각 검색기의 결과:**
- BM25 결과: A, C, B, D (키워드 일치 순)
- 임베딩 결과: A, D, B, C (의미 유사도 순)

### 1. hybrid_search (라운드 로빈 방식)
번갈아가면서 결과를 가져옵니다:
1. BM25 첫번째: A
2. 임베딩 첫번째: A (이미 있으므로 건너뜀)
3. BM25 두번째: C
4. 임베딩 두번째: D
5. BM25 세번째: B (이미 있으므로 건너뜀)

**최종 결과:** A, C, D (순서대로)

### 2. EnsembleRetriever (점수 합산 방식)
점수를 계산하고 합산합니다:
- A: BM25(0.9) + 임베딩(0.95) = 1.85
- B: BM25(0.7) + 임베딩(0.8) = 1.5
- C: BM25(0.8) + 임베딩(0.7) = 1.5
- D: BM25(0.6) + 임베딩(0.9) = 1.5

**최종 결과:** A, B/C/D (점수순)

### 핵심 차이:
- **hybrid_search**: 다양한 검색 방식의 결과를 번갈아 포함시켜 다양성 중시
- **EnsembleRetriever**: 여러 검색기에서 모두 높은 점수를 받은 문서 우선 (정확도 중시)

In [ ]:
# 복잡한 하이브리드 검색 접근법 - 커스텀 순위 재정렬
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import DocumentCompressorPipeline

# 두 검색 방법을 별도로 실행하고 결과 통합
def hybrid_search(query, k=5):
    """
    BM25와 임베딩 기반 검색을 결합한 하이브리드 검색 구현

    이 함수는 키워드 기반 검색(BM25)과 의미 기반 검색(임베딩)을 모두 수행하고
    결과를 번갈아가며 통합하여 다양한 관점의 검색 결과를 제공합니다.

    Args:
        query (str): 검색 쿼리
        k (int): 반환할 최대 문서 수

    Returns:
        List[Document]: 중복이 제거된 통합 검색 결과
    """
    # BM25 검색 결과 가져오기 (키워드 기반 검색)
    bm25_results = bm25_retriever.get_relevant_documents(query)

    # 임베딩 검색 결과 가져오기 (의미 기반 검색)
    embedding_results = chroma_retriever.get_relevant_documents(query)

    # 결과 통합 및 중복 제거를 위한 변수 초기화
    seen_contents = set()  # 이미 처리된 문서 내용을 추적
    hybrid_results = []    # 최종 결과를 저장할 리스트

    # 두 결과 세트를 번갈아가며 통합 (라운드 로빈 방식)
    for i in range(max(len(bm25_results), len(embedding_results))):
        # BM25 결과 처리 (있는 경우)
        if i < len(bm25_results):
            doc = bm25_results[i]
            # 중복 문서 확인 및 추가
            if doc.page_content not in seen_contents:
                seen_contents.add(doc.page_content)
                hybrid_results.append(doc)

        # 임베딩 검색 결과 처리 (있는 경우)
        if i < len(embedding_results):
            doc = embedding_results[i]
            # 중복 문서 확인 및 추가
            if doc.page_content not in seen_contents:
                seen_contents.add(doc.page_content)
                hybrid_results.append(doc)

        # 요청한 수(k)만큼 문서를 찾았으면 중단
        if len(hybrid_results) >= k:
            break

    # 최대 k개의 결과 반환
    return hybrid_results[:k]

In [ ]:
# 사용 예시 - 실제 쿼리로 하이브리드 검색 실행
results = hybrid_search("삼성전자의 사업영역은?", k=5)

In [ ]:
results

[Document(metadata={'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27, 'page': 14, 'page_label': '15'}, page_content='2015년말 기준 삼성전자의 총자산 금액은 약 169조에 달하며, 부채는 32.5조, 자본총계는 136조에 이르고 \n있다.'),
 Document(metadata={'creationdate': '2017-08-16T00:21:02-08:00', 'creator': 'nPDF (pdftk 1.41)', 'moddate': '2017-08-16T00:21:02-08:00', 'page': 9, 'page_label': '10', 'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27}, page_content='11\nⅡ 기업 상세 분석\n1  사업분야(내용)\nQ1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?\nA\n 삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica-\ntion) 사업부문, DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다.\n⑴ Consumer 

## 답변 얻기

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

llm_model = ChatOpenAI(model_name='gpt-4o',
                       api_key="여러분의 Key 값",
                       temperature=0)

question = "삼성전자의 사업 영역은?"

<ipython-input-40-00c7fa96e48b>:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm_model = ChatOpenAI(model_name='gpt-4o',


In [ ]:
# 앙상블 리트리버
qa = RetrievalQA.from_chain_type(
    llm = llm_model,
    chain_type='stuff',
    retriever = ensemble_retriever)

res = qa(question)
res['result']

<ipython-input-41-57bb9d34aac9>:7: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res = qa(question)


'삼성전자의 사업 영역은 크게 세 가지 부문으로 나누어져 있습니다:\n\n1. **Consumer Electronics (CE) 부문**: 이 부문에서는 TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등을 포함한 다양한 소비자 전자 제품을 생산합니다.\n\n2. **Information technology & Mobile communication (IM) 부문**: 이 부문은 주로 무선 통신 기기와 관련된 제품을 다루며, 인간 중심의 혁신을 통해 소비자들에게 새로운 가치와 편의를 제공합니다.\n\n3. **Device Solutions (DS) 부문**: 이 부문은 반도체와 같은 장치 솔루션을 포함합니다.\n\n각 부문은 독립적으로 운영되며, 삼성전자는 이러한 다양한 사업 영역을 통해 글로벌 시장에서 활동하고 있습니다.'

In [ ]:
# 크로마 리트리버
qa = RetrievalQA.from_chain_type(
    llm = llm_model,
    chain_type='stuff',
    retriever = chroma_retriever)

res = qa(question)
res['result']

'삼성전자의 사업 영역은 크게 세 가지로 나뉩니다: \n\n1. **Consumer Electronics (CE) 부문**: 이 부문에서는 TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등을 포함한 다양한 소비자 전자 제품을 생산합니다.\n\n2. **Information technology & Mobile communication (IM) 부문**: 이 부문은 주로 무선 통신 기기와 정보 기술 제품을 다루며, 인간 중심의 혁신을 통해 소비자들에게 새로운 가치와 편의를 제공합니다.\n\n3. **Device Solutions (DS) 부문**: 이 부문은 반도체와 같은 장치 솔루션을 제공하는 데 중점을 둡니다. \n\n각 부문은 독립적으로 운영되며, 삼성전자의 다양한 제품과 기술 혁신을 통해 시장에서 경쟁력을 유지하고 있습니다.'

In [ ]:
# BM25 리트리버
qa = RetrievalQA.from_chain_type(
    llm = llm_model,
    chain_type='stuff',
    retriever = bm25_retriever)

res = qa(question)
res['result']

'삼성전자의 사업 영역은 주로 모바일, 가전, 그리고 최근에는 자동차 전장(電裝) 사업으로 확장되고 있습니다. 특히, Connected Car 시장에 진출하기 위해 미국의 전장업체인 Harman을 인수하는 등 자동차 관련 기술 개발에도 집중하고 있습니다.'

하이브리드 검색을 RetrievalQA에 사용하려면 먼저 커스텀 Retriever 클래스를 만들어야 합니다.

In [ ]:
from langchain.schema.retriever import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from typing import List, Any
from pydantic import Field

class HybridSearchRetriever(BaseRetriever):
    """하이브리드 검색(BM25 + 임베딩)을 수행하는 커스텀 리트리버"""

    bm25_retriever: Any = Field(description="BM25 검색기")
    embedding_retriever: Any = Field(description="임베딩 검색기")
    k: int = Field(default=5, description="반환할 최대 문서 수")

    class Config:
        arbitrary_types_allowed = True

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun = None
    ) -> List[Document]:
        """하이브리드 검색 수행"""
        # 각 검색기에서 결과 가져오기
        bm25_results = self.bm25_retriever.get_relevant_documents(query)
        embedding_results = self.embedding_retriever.get_relevant_documents(query)

        # 결과 통합 및 중복 제거
        seen_contents = set()
        hybrid_results = []

        # 두 결과 세트를 번갈아가며 통합
        for i in range(max(len(bm25_results), len(embedding_results))):
            if i < len(bm25_results):
                doc = bm25_results[i]
                if doc.page_content not in seen_contents:
                    seen_contents.add(doc.page_content)
                    hybrid_results.append(doc)

            if i < len(embedding_results):
                doc = embedding_results[i]
                if doc.page_content not in seen_contents:
                    seen_contents.add(doc.page_content)
                    hybrid_results.append(doc)

            if len(hybrid_results) >= self.k:
                break

        return hybrid_results[:self.k]

In [ ]:
# 하이브리드 검색 리트리버 생성
hybrid_retriever = HybridSearchRetriever(
    bm25_retriever=bm25_retriever,
    embedding_retriever=chroma_retriever,
    k=5
)

# RetrievalQA 체인 생성
qa = RetrievalQA.from_chain_type(
    llm=llm_model,
    chain_type='stuff',
    retriever=hybrid_retriever
)

# 질문 실행
question = "삼성전자의 사업 영역은?"
res = qa(question)
print(res['result'])

삼성전자의 사업 영역은 크게 세 가지 부문으로 나누어져 있습니다:

1. **Consumer Electronics (CE) 부문**: 이 부문에서는 TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등을 포함한 다양한 소비자 전자 제품을 생산합니다.

2. **Information technology & Mobile communication (IM) 부문**: 이 부문은 주로 무선 통신 기기와 관련된 제품을 다루며, 인간 중심의 혁신을 통해 소비자들에게 새로운 가치와 편의를 제공합니다.

3. **Device Solutions (DS) 부문**: 이 부문은 반도체와 같은 장치 솔루션을 포함합니다.

각 부문은 독립적으로 운영되며, 삼성전자는 이러한 다양한 사업 영역을 통해 글로벌 시장에서 활동하고 있습니다.
